In [1]:
import pandas as pd
from Bio import Medline
from Bio import Entrez
import string

Entrez.email = "1219131962@qq.com"
Entrez.api_key = "6fcb01c2989c35aa1da3d9d0118abedfc409"
PMID_info = []
CGM_result_dict = {}


def get_keywords(PMID_list):
    """Thank to Dr. Mao Zhitao for his help in this function."""
    
    for i in PMID_list:
        # print(i)
        handle = Entrez.efetch(db="pubmed", id=i, rettype="medline", retmode="text")
        records = Medline.parse(handle)
        records = list(records)

        for record in records:
            title = record.get("TI", "?")
            abstract = record.get("AB", "?")
            keywords = record.get("OT", "?")
            Journal = record.get("JT", "?")
            country = record.get("AD", "?")[0].split(", ")[-1].strip(string.punctuation)
            pmid = record.get("PMID", "?")
            Author = record.get("FAU", "?")
            doi = record.get("SO", "?")
            date = record.get("DEP", "?")
            year = record.get("DP", "?")
            institution = record.get("AD", "?")[0]

            PMID_info.append(
                {
                    "PMID": pmid,
                    "DOI": doi,
                    "Author": Author,
                    "Title": title,
                    "Abstract": abstract,
                    "Keywords": keywords,
                    "Journal": Journal,
                    "Institution": institution,
                    "Country": country,
                    "Date": date,
                    "Year": year,
                }
            )

    for key, value in CGM_result_dict.items():
        PMID_info.append([key, value])

    return PMID_info

先去pubmed上根据菌种的名字，检索下载所有发表的csv文件

In [2]:
"""
测试文献解析代码
"""
df = pd.read_csv('pmid-demo.txt', sep='\t', header=None, names=['PMID'])

# 删除重复值
df = df.dropna(subset=['PMID'])
df = df.reset_index(drop=True)

get_keywords(list(df.iloc[:, 0]))
df_info = pd.DataFrame(PMID_info)

df_info['DOI'] = df_info['DOI'].str.extract(r'(10\.\d{4,9}\/[-._;()/:A-Za-z0-9]+)')
df_info['DOI'] = df_info['DOI'].str.rstrip('.')

In [3]:
df_info

,PMID,DOI,Author,Title,Abstract,Keywords,Journal,Institution,Country,Date,Year
0,25025273,10.3109/07388551.2014.923985,"[Singh, Bijender]",Myceliophthora thermophila syn. Sporotrichum t...,Myceliophthora thermophila syn. Sporotrichum t...,"[Biomolecules, Myceliophthora thermophila, Spo...",Critical reviews in biotechnology,"a Laboratory of Bioprocess Technology, Departm...",India,20140715,2016
1,32640074,10.1111/jam.14774,"[Dos Santos Gomes, A C, Casciatori, F P, Gomes...",Growth kinetics of Myceliophthora thermophila ...,AIMS: This work aimed to estimate the growth o...,"[N-acetylglucosamine, enzymes, growth kinetics...",Journal of applied microbiology,"Instituto de Biociencias, Letras e Ciencias Ex...",Brazil,20200729,2021 Jan


In [74]:
df_Mt1 = pd.read_csv("./csv-Myceliopht-set.csv")
df_Mt1

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,25025273,Myceliophthora thermophila syn. Sporotrichum t...,Singh B.,Crit Rev Biotechnol. 2016;36(1):59-69. doi: 10...,Singh B,Crit Rev Biotechnol,2016,2014/07/16,NaN,NaN,10.3109/07388551.2014.923985
1,32640074,Growth kinetics of Myceliophthora thermophila ...,"Dos Santos Gomes AC, Casciatori FP, Gomes E, d...",J Appl Microbiol. 2021 Jan;130(1):90-99. doi: ...,Dos Santos Gomes AC,J Appl Microbiol,2021,2020/07/09,NaN,NaN,10.1111/jam.14774
2,24995002,Genomic insights into the fungal lignocellulol...,"Karnaouri A, Topakas E, Antonopoulou I, Christ...",Front Microbiol. 2014 Jun 18;5:281. doi: 10.33...,Karnaouri A,Front Microbiol,2014,2014/07/05,PMC4061905,NaN,10.3389/fmicb.2014.00281
3,35450635,Extending the diversity of Myceliophthora ther...,"Sun P, de Munnik M, van Berkel WJH, Kabel MA.",Carbohydr Polym. 2022 Jul 15;288:119373. doi: ...,Sun P,Carbohydr Polym,2022,2022/04/22,NaN,NaN,10.1016/j.carbpol.2022.119373
4,31534479,Myceliophthora thermophila Xyr1 is predominant...,"Dos Santos Gomes AC, Falkoski D, Battaglia E, ...",Biotechnol Biofuels. 2019 Sep 16;12:220. doi: ...,Dos Santos Gomes AC,Biotechnol Biofuels,2019,2019/09/20,PMC6745793,NaN,10.1186/s13068-019-1556-y
...,...,...,...,...,...,...,...,...,...,...,...
261,30383933,[CHARACTERISTICS OF FUNGAL STRAINS PRODUCING T...,"Krestyanova IN, Sakhibgaraeva LF, Berezina OV,...",Mol Gen Mikrobiol Virusol. 2016;34(3):109-114.,Krestyanova IN,Mol Gen Mikrobiol Virusol,2016,2018/11/02,NaN,NaN,NaN
262,18668239,Laccase-induced C-N coupling of substituted p-...,"Mikolasch A, Matthies A, Lalk M, Schauer F.",Appl Microbiol Biotechnol. 2008 Sep;80(3):389-...,Mikolasch A,Appl Microbiol Biotechnol,2008,2008/08/01,NaN,NaN,10.1007/s00253-008-1595-y
263,36866191,Safety and efficacy of a feed additive consist...,EFSA Panel on Additives and Products or Substa...,EFSA J. 2023 Feb 28;21(2):e07873. doi: 10.2903...,EFSA Panel on Additives and Products or Substa...,EFSA J,2023,2023/03/03,PMC9972018,NaN,10.2903/j.efsa.2023.7873
264,37164048,Production of recombinant lytic polysaccharide...,"Gonçalves AL, Cunha PM, da Silva Lima A, Dos S...",Biochim Biophys Acta Proteins Proteom. 2023 Ma...,Gonçalves AL,Biochim Biophys Acta Proteins Proteom,2023,2023/05/10,NaN,NaN,10.1016/j.bbapap.2023.140919


In [75]:
df_Mt2 = pd.read_csv("./csv-Thermothel-set.csv")
df_Mt2

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,35205873,Engineered Fungus Thermothelomyces thermophilu...,"Balabanova L, Seitkalieva A, Yugay Y, Rusapeto...",J Fungi (Basel). 2022 Jan 26;8(2):119. doi: 10...,Balabanova L,J Fungi (Basel),2022,2022/02/25,PMC8877005,NaN,10.3390/jof8020119
1,31309856,Isolation and characterization of a new cytoto...,"Gao YL, Zhang ML, Wang X, Zhang HD, Huang JZ, ...",Nat Prod Res. 2021 Jun;35(11):1792-1798. doi: ...,Gao YL,Nat Prod Res,2021,2019/07/17,NaN,NaN,10.1080/14786419.2019.1641810
2,36506193,Identification of Thermostable Xylose Reductas...,"Ali N, Aiman A, Shamsi A, Hassan I, Shahid M, ...",ACS Omega. 2022 Nov 17;7(48):44241-44250. doi:...,Ali N,ACS Omega,2022,2022/12/12,PMC9730754,NaN,10.1021/acsomega.2c05690
3,33461124,Fed-batch production of Thermothelomyces therm...,"Liu E, Segato F, Wilkins MR.",Bioresour Technol. 2021 Apr;325:124700. doi: 1...,Liu E,Bioresour Technol,2021,2021/01/18,NaN,NaN,10.1016/j.biortech.2021.124700
4,32653619,Enzymatic versatility and thermostability of a...,"Kadowaki MAS, Higasi PMR, de Godoy MO, de Araú...",Biochim Biophys Acta Gen Subj. 2020 Oct;1864(1...,Kadowaki MAS,Biochim Biophys Acta Gen Subj,2020,2020/07/13,NaN,NaN,10.1016/j.bbagen.2020.129681
5,33301505,Safety evaluation of a β-mannanase enzyme prep...,"Kern A, Shanahan D, Buesen R, Geiger D.",PLoS One. 2020 Dec 10;15(12):e0243647. doi: 10...,Kern A,PLoS One,2020,2020/12/10,PMC7728267,NaN,10.1371/journal.pone.0243647
6,33712158,Light-stimulated T. thermophilus two-domain LP...,"Higasi PMR, Velasco JA, Pellegrini VOA, de Ara...",Carbohydr Polym. 2021 May 15;260:117814. doi: ...,Higasi PMR,Carbohydr Polym,2021,2021/03/13,NaN,NaN,10.1016/j.carbpol.2021.117814
7,34153346,Survival of thermophilic fungi in various pres...,Berikten D.,Cryobiology. 2021 Aug;101:38-43. doi: 10.1016/...,Berikten D,Cryobiology,2021,2021/06/21,NaN,NaN,10.1016/j.cryobiol.2021.06.005
8,34560965,Unique features of the bifunctional GH30 from ...,"Nikolaivits E, Pentari C, Kosinas C, Feiler CG...",Carbohydr Polym. 2021 Dec 1;273:118553. doi: 1...,Nikolaivits E,Carbohydr Polym,2021,2021/09/25,NaN,NaN,10.1016/j.carbpol.2021.118553
9,32380451,FTacV study of electroactive immobilized enzym...,"Zouraris D, Kiafi S, Zerva A, Topakas E, Karan...",Bioelectrochemistry. 2020 Aug;134:107538. doi:...,Zouraris D,Bioelectrochemistry,2020,2020/05/08,NaN,NaN,10.1016/j.bioelechem.2020.107538


In [76]:
"""
嗜热毁丝霉有两个常用名称，先聚合，然后去重
- 结果显示，Mt这个名字的csv，包含了Tt的.....
"""

# 合并两个dataframe，如果PMID有重复的，则保留第一个
df_Mt = pd.concat([df_Mt1, df_Mt2], ignore_index=True)
df_Mt = df_Mt.drop_duplicates(subset=["PMID"], keep="first")

# 取df_Mt中的PMID和DOI列
df_Mt_filter = df_Mt[["PMID", "DOI", "Publication Year"]]

# 输出DOI为空的PMID
# df_Mt_filter[df_Mt_filter['DOI'].isnull()]

# 将DOI的缺失值填充为空
# df_Mt_filter.loc[:, 'DOI'] = df_Mt_filter['DOI'].fillna('')
df_Mt_filter = df_Mt_filter.copy()
df_Mt_filter["DOI"] = df_Mt_filter["DOI"].fillna("")

# 删除PMID列中的重复值
df_Mt_filter = df_Mt_filter.drop_duplicates(subset=["PMID"], keep="first")

In [80]:
"""
get_keywords这个函数一般一次最好处理100个PMID，否则容易被封IP
"""

# 将PMID转换为list
PMID_list = list(df_Mt_filter["PMID"])

# 输出PMID的数量
len(PMID_list)

266

In [81]:
# 将PMID分为多个list，每个list最多100个PMID
PMID_list_1 = PMID_list[:100]
PMID_list_2 = PMID_list[100:200]
PMID_list_3 = PMID_list[200:300]

In [82]:
df_list_1 = pd.DataFrame(get_keywords(PMID_list_1))

In [83]:
df_list_2 = pd.DataFrame(get_keywords(PMID_list_2))

In [84]:
df_list_3 = pd.DataFrame(get_keywords(PMID_list_3))

In [130]:
# 如果PMID有重复的，则保留第一个
df = df_list_3.drop_duplicates(subset=["PMID"], keep="first")

In [131]:
# 创建新列
df["DOI"] = ""
df["Publication Year"] = ""

# 重新排序
df = df[
    [
        "PMID",
        "Author",
        "Title",
        "Abstract",
        "Keywords",
        "Journal",
        "Institution",
        "Country",
        "DOI",
        "Publication Year",
    ]
]

df.head()

,PMID,Author,Title,Abstract,Keywords,Journal,Institution,Country,DOI,Publication Year
0,25025273,"[Singh, Bijender]",Myceliophthora thermophila syn. Sporotrichum t...,Myceliophthora thermophila syn. Sporotrichum t...,"[Biomolecules, Myceliophthora thermophila, Spo...",Critical reviews in biotechnology,"a Laboratory of Bioprocess Technology, Departm...",India,,
1,32640074,"[Dos Santos Gomes, A C, Casciatori, F P, Gomes...",Growth kinetics of Myceliophthora thermophila ...,AIMS: This work aimed to estimate the growth o...,"[N-acetylglucosamine, enzymes, growth kinetics...",Journal of applied microbiology,"Instituto de Biociencias, Letras e Ciencias Ex...",Brazil,,
2,24995002,"[Karnaouri, Anthi, Topakas, Evangelos, Antonop...",Genomic insights into the fungal lignocellulol...,The microbial conversion of solid cellulosic b...,"[CAZy, Myceliophthora thermophila, biofuels, l...",Frontiers in microbiology,"Biotechnology Laboratory, Department of Synthe...",Sweden,,
3,35450635,"[Sun, Peicheng, de Munnik, Melanie, van Berkel...",Extending the diversity of Myceliophthora ther...,Lytic polysaccharide monooxygenases (LPMOs) pl...,"[Active site segment, LPMOs, Lignocellulose, M...",Carbohydrate polymers,"Laboratory of Food Chemistry, Wageningen Unive...",the Netherlands. Electronic address: peicheng....,,
4,31534479,"[Dos Santos Gomes, Ana Carolina, Falkoski, Dan...",Myceliophthora thermophila Xyr1 is predominant...,BACKGROUND: Myceliophthora thermophila is a th...,"[Cellulose degradation, Myceliophthora thermop...",Biotechnology for biofuels,"1Fungal Physiology, Westerdijk Fungal Biodiver...",The Netherlands. ISNI: 0000000120346234. GRID:...,,


In [132]:
# 将PMID的值转换为str
df_Mt_filter["PMID"] = df_Mt_filter["PMID"].astype(str)

df_Mt_filter

,PMID,DOI,Publication Year
0,25025273,10.3109/07388551.2014.923985,2016
1,32640074,10.1111/jam.14774,2021
2,24995002,10.3389/fmicb.2014.00281,2014
3,35450635,10.1016/j.carbpol.2022.119373,2022
4,31534479,10.1186/s13068-019-1556-y,2019
...,...,...,...
261,30383933,,2016
262,18668239,10.1007/s00253-008-1595-y,2008
263,36866191,10.2903/j.efsa.2023.7873,2023
264,37164048,10.1016/j.bbapap.2023.140919,2023


In [133]:
# 将df中的DOI和Publication Year填充
for i in range(len(df)):
    for j in range(len(df_Mt_filter)):
        if df.loc[i, "PMID"] == df_Mt_filter.loc[j, "PMID"]:
            doi = df_Mt_filter.loc[j, "DOI"]
            pub_year = df_Mt_filter.loc[j, "Publication Year"]
            if pd.isnull(doi):
                doi = ""
            if pd.isnull(pub_year):
                pub_year = ""
            df.loc[i, "DOI"] = doi
            df.loc[i, "Publication Year"] = pub_year

In [134]:
df

,PMID,Author,Title,Abstract,Keywords,Journal,Institution,Country,DOI,Publication Year
0,25025273,"[Singh, Bijender]",Myceliophthora thermophila syn. Sporotrichum t...,Myceliophthora thermophila syn. Sporotrichum t...,"[Biomolecules, Myceliophthora thermophila, Spo...",Critical reviews in biotechnology,"a Laboratory of Bioprocess Technology, Departm...",India,10.3109/07388551.2014.923985,2016
1,32640074,"[Dos Santos Gomes, A C, Casciatori, F P, Gomes...",Growth kinetics of Myceliophthora thermophila ...,AIMS: This work aimed to estimate the growth o...,"[N-acetylglucosamine, enzymes, growth kinetics...",Journal of applied microbiology,"Instituto de Biociencias, Letras e Ciencias Ex...",Brazil,10.1111/jam.14774,2021
2,24995002,"[Karnaouri, Anthi, Topakas, Evangelos, Antonop...",Genomic insights into the fungal lignocellulol...,The microbial conversion of solid cellulosic b...,"[CAZy, Myceliophthora thermophila, biofuels, l...",Frontiers in microbiology,"Biotechnology Laboratory, Department of Synthe...",Sweden,10.3389/fmicb.2014.00281,2014
3,35450635,"[Sun, Peicheng, de Munnik, Melanie, van Berkel...",Extending the diversity of Myceliophthora ther...,Lytic polysaccharide monooxygenases (LPMOs) pl...,"[Active site segment, LPMOs, Lignocellulose, M...",Carbohydrate polymers,"Laboratory of Food Chemistry, Wageningen Unive...",the Netherlands. Electronic address: peicheng....,10.1016/j.carbpol.2022.119373,2022
4,31534479,"[Dos Santos Gomes, Ana Carolina, Falkoski, Dan...",Myceliophthora thermophila Xyr1 is predominant...,BACKGROUND: Myceliophthora thermophila is a th...,"[Cellulose degradation, Myceliophthora thermop...",Biotechnology for biofuels,"1Fungal Physiology, Westerdijk Fungal Biodiver...",The Netherlands. ISNI: 0000000120346234. GRID:...,10.1186/s13068-019-1556-y,2019
...,...,...,...,...,...,...,...,...,...,...
261,30383933,"[Krestyanova, I N, Sakhibgaraeva, L F, Berezin...",[CHARACTERISTICS OF FUNGAL STRAINS PRODUCING T...,Fungal strains degrading plant biomass availab...,?,"Molekuliarnaia genetika, mikrobiologiia i viru...",?,,,2016
262,18668239,"[Mikolasch, Annett, Matthies, Anastasia, Lalk,...",Laccase-induced C-N coupling of substituted p-...,Fungal laccases (benzenediol:oxygen oxidoreduc...,?,Applied microbiology and biotechnology,"Institute of Microbiology, Ernst-Moritz-Arndt-...",Germany. annett.mikolasch@uni-greifswald.de,10.1007/s00253-008-1595-y,2008
263,36866191,"[Bampidis, Vasileios, Azimonti, Giovanna, Bast...",Safety and efficacy of a feed additive consist...,Following a request from the European Commissi...,"[Natupulse(R), digestibility enhancers, effica...",EFSA journal. European Food Safety Authority,?,,10.2903/j.efsa.2023.7873,2023
264,37164048,"[Goncalves, Aline Larissa, Cunha, Paula Macedo...",Production of recombinant lytic polysaccharide...,Lignocellulosic biomass is a promising alterna...,"[Enzymes, Filamentous fungi, LPMO, Saccharific...",Biochimica et biophysica acta. Proteins and pr...,"Department of Biotechnology, Lorena School of ...",Brazil,10.1016/j.bbapap.2023.140919,2023


In [4]:
# 创建一个副本，免得操作失误
df_copy = df.copy()
df_copy

NameError: name 'df' is not defined

In [153]:
# Keywords列中的值转换为str
df_copy["Keywords"] = df_copy["Keywords"].astype(str)

# 判断Keywords列的值是否为?
for i in range(len(df_copy)):
    if df_copy.loc[i, "Keywords"] == "?":
        # 将?替换为空
        df_copy.loc[i, "Keywords"] = ""
    else:
        # 去掉Keywords列中的[]
        df_copy.loc[i, "Keywords"] = df_copy.loc[i, "Keywords"].strip("[]")
        # 将单引号替换为空
        df_copy.loc[i, "Keywords"] = df_copy.loc[i, "Keywords"].replace("'", "")

df_copy

,PMID,Author,Title,Abstract,Keywords,Journal,Institution,Country,DOI,Publication Year
0,25025273,"[Singh, Bijender]",Myceliophthora thermophila syn. Sporotrichum t...,Myceliophthora thermophila syn. Sporotrichum t...,"Biomolecules, Myceliophthora thermophila, Spor...",Critical reviews in biotechnology,"a Laboratory of Bioprocess Technology, Departm...",India,10.3109/07388551.2014.923985,2016
1,32640074,"[Dos Santos Gomes, A C, Casciatori, F P, Gomes...",Growth kinetics of Myceliophthora thermophila ...,AIMS: This work aimed to estimate the growth o...,"N-acetylglucosamine, enzymes, growth kinetics,...",Journal of applied microbiology,"Instituto de Biociencias, Letras e Ciencias Ex...",Brazil,10.1111/jam.14774,2021
2,24995002,"[Karnaouri, Anthi, Topakas, Evangelos, Antonop...",Genomic insights into the fungal lignocellulol...,The microbial conversion of solid cellulosic b...,"CAZy, Myceliophthora thermophila, biofuels, li...",Frontiers in microbiology,"Biotechnology Laboratory, Department of Synthe...",Sweden,10.3389/fmicb.2014.00281,2014
3,35450635,"[Sun, Peicheng, de Munnik, Melanie, van Berkel...",Extending the diversity of Myceliophthora ther...,Lytic polysaccharide monooxygenases (LPMOs) pl...,"Active site segment, LPMOs, Lignocellulose, Ma...",Carbohydrate polymers,"Laboratory of Food Chemistry, Wageningen Unive...",the Netherlands. Electronic address: peicheng....,10.1016/j.carbpol.2022.119373,2022
4,31534479,"[Dos Santos Gomes, Ana Carolina, Falkoski, Dan...",Myceliophthora thermophila Xyr1 is predominant...,BACKGROUND: Myceliophthora thermophila is a th...,"Cellulose degradation, Myceliophthora thermoph...",Biotechnology for biofuels,"1Fungal Physiology, Westerdijk Fungal Biodiver...",The Netherlands. ISNI: 0000000120346234. GRID:...,10.1186/s13068-019-1556-y,2019
...,...,...,...,...,...,...,...,...,...,...
261,30383933,"[Krestyanova, I N, Sakhibgaraeva, L F, Berezin...",[CHARACTERISTICS OF FUNGAL STRAINS PRODUCING T...,Fungal strains degrading plant biomass availab...,,"Molekuliarnaia genetika, mikrobiologiia i viru...",?,,,2016
262,18668239,"[Mikolasch, Annett, Matthies, Anastasia, Lalk,...",Laccase-induced C-N coupling of substituted p-...,Fungal laccases (benzenediol:oxygen oxidoreduc...,,Applied microbiology and biotechnology,"Institute of Microbiology, Ernst-Moritz-Arndt-...",Germany. annett.mikolasch@uni-greifswald.de,10.1007/s00253-008-1595-y,2008
263,36866191,"[Bampidis, Vasileios, Azimonti, Giovanna, Bast...",Safety and efficacy of a feed additive consist...,Following a request from the European Commissi...,"Natupulse(R), digestibility enhancers, efficac...",EFSA journal. European Food Safety Authority,?,,10.2903/j.efsa.2023.7873,2023
264,37164048,"[Goncalves, Aline Larissa, Cunha, Paula Macedo...",Production of recombinant lytic polysaccharide...,Lignocellulosic biomass is a promising alterna...,"Enzymes, Filamentous fungi, LPMO, Saccharifica...",Biochimica et biophysica acta. Proteins and pr...,"Department of Biotechnology, Lorena School of ...",Brazil,10.1016/j.bbapap.2023.140919,2023


In [1]:
# df_copy按照Year列进行降序排列
df_copy = df_copy.sort_values(by="Year", ascending=False)
df_copy

NameError: name 'df_copy' is not defined

In [154]:
df_copy.to_excel("Mt_paper_all.xlsx", index=False)